In [ ]:
import os
import string
import glob
from tensorflow.keras.applications.inception_v3 import InceptionV3
import tensorflow.keras.applications.inception_v3
from random import randint
from tqdm import tqdm
import tensorflow.keras.preprocessing.image
import pickle
from time import time
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (LSTM,Embedding,TimeDistributed,Dense,RepeatVector,Activation,Flatten,Reshape,concatenate,Dropout,BatchNormalization)
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras import Input,layers
from tensorflow.keras import optimizers
from random import random
from tensorflow.keras.models import Model
import nltk
from tensorflow.keras.layers import add
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
root_captioning="/content/gdrive/My Drive/projects/captions/"

In [ ]:
#READ THE CAPTIONS FILE INTO A LIST
captions=open('/content/gdrive/My Drive/projects/captions/Flickr8k_text/Flickr8k.lemma.token1.txt').read().split("\n")

In [ ]:
#MAKE A DICTIONARY OF ALL THE CAPTIONS. 
pre_processed_captions=dict()
for i in captions:
  tok=i.split()
  if len(i) >= 2:
    id=tok[0].split('.')[0]
    description=tok[1:]
  
    description=[i1.lower() for i1 in description]
    description=[i1.translate(str.maketrans('', '', string.punctuation)) for i1 in description]
    description=[i1 for i1 in description if i1.isalpha()]

    if id not in pre_processed_captions:
      pre_processed_captions[id]=list()
    pre_processed_captions[id].append(' '.join(description))


In [ ]:
#MAKE THE LISTS OF ALL THE CAPTIONS.
captions_list=[]
for i in captions:
  tok=i.split()
  if len(i)>=2:
    desc=tok[1:]
    captions_list.append(desc)

In [ ]:
#FIND THE LENGTH OF THE LONGEST CAPTION.
max_len=0
for k,v in pre_processed_captions.items():
  for i in v:
    tok=i.split()
    if len(tok)>max_len:
      max_len=len(tok)
      max_tok=tok

In [ ]:
#FIND ALL THE UNIQUE WORDS FROM THE CAPTION_LIST.
unique_words=set()
for key in pre_processed_captions:
  for i in pre_processed_captions[key]:
    unique_words.update(i.split())

In [ ]:
#LIST OF PATHS OF ALL THE IMAGES.
image_path_list=[]
for key,value in pre_processed_captions.items():
  str=os.path.join(root_captioning,'Flicker8k_Dataset',key)
  str=str+'.jpg'
  image_path_list.append(str)

In [ ]:
#MAKE LIST OF THE IMAGE NAMES OF ALL THE IMAGES WHOSE CAPTIONS EXISTS.
k_list=[]
for k,v in pre_processed_captions.items():
  tok=k.split('.')
  k_list.append(tok)

key_list = [item for sublist in k_list for item in sublist]  

In [ ]:
#MAKE LIST OF WHOLE IMAGE NAMES USED FOR COMPARISON IN DIVISION OF TRAIN AND TEST IF IMAGE NAME EXISTS
k_list1=[]
for k,v in pre_processed_captions.items():
  k_list1.append(k+'.jpg')  

In [ ]:
#MAKE LIST OF TRAIN AND TEST IMAGES WHOSE CAPTIONS EXISTS IN PRE_PROCESSED_CAPTIONS DICTIONARY
train_list=[]
test_list=[]

train_img_list=open('/content/gdrive/My Drive/projects/captions/Flickr8k_text/Flickr_8k.trainImages.txt').read().split("\n")
for i in train_img_list:
  if i in k_list1:
    train_list.append(i)

test_img_list=open('/content/gdrive/My Drive/projects/captions/Flickr8k_text/Flickr_8k.testImages.txt').read().split("\n")
for i in test_img_list:
  if i in k_list1:
    test_list.append(i)

In [ ]:
images_path='/content/gdrive/MyDrive/projects/captions/Flicker8k_Dataset/'
test_img_path_list=[]
for item in test_list:
  img_path=os.path.join(images_path,item,'.npy')
  test_img_path_list.append(img_path)

In [ ]:
#MAKE DICTIONARY FROM PREPROCESSED CAPTIONS THAT ONLY CONTAINS TRAINING IMAGE CAPTIONS
training=[]
for i in train_img_list:
  id=i.split('.')[0]
  training.append(id)

training_dictionary_captions=dict()
for key,value in pre_processed_captions.items():
  for i in training:
    if key in i:
      training_dictionary_captions.update({key:value})

In [ ]:
#ADD START OF SEQUENCE AND END OF SEQUENCE TOKEN TO EACH CAPTION OF TRAINING DICTIONARY CAPTION FOR THE LSTM TO TRAIN
START="startsequence"
END="endsequence"
for key,value in training_dictionary_captions.items():
  for k in range(len(value)):
    value[k]=f'{START} {value[k]} {END}'

In [ ]:
whole_model = InceptionV3(weights='imagenet')
encode_model = Model(whole_model.input,whole_model.layers[-2].output)

96116736/96112376 [==============================] - 2s 0us/step


In [ ]:
encode_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
#ENCODE THE TRAINING IMAGES
encoding_train = {}
for id in tqdm(train_list):
  image_path = os.path.join(root_captioning,'Flicker8k_Dataset', id)
  img = tensorflow.keras.preprocessing.image.load_img(image_path,target_size=(299,299))
  img=img.resize((299,299),Image.ANTIALIAS)
  x=tensorflow.keras.preprocessing.image.img_to_array(img)
  x=np.expand_dims(x,axis=0)
  x=tensorflow.keras.applications.inception_v3.preprocess_input(x)
  x=encode_model.predict(x)
  x=np.reshape(x,2048)
  encoding_train[id] = x

100%|██████████| 6000/6000 [34:04<00:00,  2.94it/s]


In [ ]:
#ENCODE THE TESTING IMAGES
encoding_test = {}
for id in tqdm(test_list):
  image_path = os.path.join(root_captioning,'Flicker8k_Dataset', id)
  img = tensorflow.keras.preprocessing.image.load_img(image_path,target_size=(299,299))
  img=img.resize((299,299),Image.ANTIALIAS)
  x=tensorflow.keras.preprocessing.image.img_to_array(img)
  x=np.expand_dims(x,axis=0)
  x=tensorflow.keras.applications.inception_v3.preprocess_input(x)
  x=encode_model.predict(x)
  x=np.reshape(x,2048)
  encoding_test[id] = x

100%|██████████| 1000/1000 [05:55<00:00,  2.81it/s]


In [ ]:
with open("/content/gdrive/My Drive/lstm/train2048.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_train, encoded_pickle)

In [ ]:
with open("/content/gdrive/My Drive/lstm/test2048.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_test, encoded_pickle)

In [ ]:
encoding_train = pickle.load(open("/content/gdrive/My Drive/lstm/train2048.pkl", "rb"))
encoding_test = pickle.load(open("/content/gdrive/My Drive/lstm/test2048.pkl", "rb"))

In [ ]:
#MAKE A LIST OF ALL THE SENTENCES IN THE TRAINING CAPTIONS
all_train=list(training_dictionary_captions.values())
all_training_captions = [item for sublist in all_train for item in sublist]

In [ ]:
#MAKE A LIST OF ALL THE UNIQUE WORDS
words=[]
for sentences in all_training_captions:
    for word in sentences.split(' '):
        words.append(word)

word_list=list(set(words))

In [ ]:
#MAKE DICTIONARY OF UNIQUE WORDS
idxtoword={}
wordtoidx={}

ix=1
for word in word_list:
  wordtoidx[word]=ix
  idxtoword[ix]=word
  ix+=1

vocab_size = len(idxtoword) + 1 
vocab_size

5773

In [ ]:
#print maximum length of the padding
max_len +=2
print(max_len)

37


In [ ]:
def data_generator(batch_size):
  # x1 - Training data for photos
  # x2 - The caption that goes with each photo
  # y - The predicted rest of the caption
  train_photo=[]
  caption_photo=[]
  caption_predicted=[]
  n=0
  while True:
    for key, cap_list in training_dictionary_captions.items():
      n+=1
      photo = encoding_train[key+'.jpg']
      for captions in cap_list:
        seq=[wordtoidx[word] for word in captions.split(' ') if word in wordtoidx]
        for i in range(1,len(seq)):
          in_seq,out_seq = seq[:i],seq[i]
          in_seq = pad_sequences([in_seq],maxlen=max_len)[0]
          out_seq = to_categorical([out_seq],num_classes=vocab_size)[0]
          train_photo.append(photo)
          caption_photo.append(in_seq)
          caption_predicted.append(out_seq)
      if n==batch_size:
        yield([np.array(train_photo),np.array(caption_photo)],np.array(caption_predicted))
        train_photo=[]
        caption_photo=[]
        caption_predicted=[]
        n=0

In [ ]:
glove_dir = os.path.join(root_captioning,'glove.6B')
embeddings_index = {} 
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print(f'Found {len(embeddings_index)} word vectors.')

400000it [00:22, 17694.04it/s]

Found 400000 word vectors.


In [ ]:
embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoidx.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [ ]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(300)(fe1)
fe3 = Dense(256, activation='relu')(fe2)
inputs2 = Input(shape=(max_len,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = Dense(300)(se2)
se4 = LSTM(256)(se3)
decoder1 = add([fe3, se4])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
caption_model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [ ]:
caption_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 37)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 37, 200)      1154600     input_3[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_2[0][0]                    
_______________________________________________________________________________________

In [ ]:
caption_model.layers[2].set_weights([embedding_matrix])
caption_model.layers[2].trainable = False

def scheduler(epoch, lr):
  if epoch < 10:
    return learning_rate
  elif epoch<20:
    return learning_rate * tensorflow.math.exp(0.1)
  else:
    return learning_rate * tensorflow.math.exp(0.2)

adam=tensorflow.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, 
                                      epsilon=1e-07, amsgrad=False,name='Adam')

caption_model.compile(loss='categorical_crossentropy', optimizer=adam,metrics=['accuracy'])

In [ ]:
batch_size = 5
steps = len(all_training_captions)//batch_size

In [ ]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
generator = data_generator(batch_size)
caption_model.fit(generator, epochs=30,steps_per_epoch=steps, callbacks=[callback], verbose=1)

Epoch 1/30
6000/6000 [==============================] - 624s 104ms/step - loss: 3.7075 - accuracy: 0.3141
Epoch 2/30
6000/6000 [==============================] - 641s 107ms/step - loss: 2.9851 - accuracy: 0.3740
Epoch 3/30
6000/6000 [==============================] - 655s 109ms/step - loss: 2.7164 - accuracy: 0.3979
Epoch 4/30
6000/6000 [==============================] - 658s 110ms/step - loss: 2.5574 - accuracy: 0.4160
Epoch 5/30
6000/6000 [==============================] - 647s 108ms/step - loss: 2.4509 - accuracy: 0.4295
Epoch 6/30
6000/6000 [==============================] - 645s 107ms/step - loss: 2.3688 - accuracy: 0.4397
Epoch 7/30
6000/6000 [==============================] - 648s 108ms/step - loss: 2.3039 - accuracy: 0.4482
Epoch 8/30
6000/6000 [==============================] - 646s 108ms/step - loss: 2.2481 - accuracy: 0.4558
Epoch 9/30
6000/6000 [==============================] - 644s 107ms/step - loss: 2.2014 - accuracy: 0.4619
Epoch 10/30
6000/6000 [=======================

In [ ]:
caption_model.save_weights('/content/gdrive/My Drive/lstm/caption-model.hdf5')

In [ ]:
def generateCaption(photo):
    in_text = START
    for i in range(max_len):
        sequence = [wordtoidx[w] for w in in_text.split() if w in wordtoidx]
        sequence = pad_sequences([sequence], maxlen=max_len)
        yhat = caption_model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = idxtoword[yhat]
        in_text += ' ' + word
        if word == END:
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [3]:
#CALCULATING BLEU SCORE FOR 10 RANDOMLY GENERATED IMAGES AND VIEWING IT
for z in range(10): # set higher to see more examples
  rand1=randint(1,1000)
  pic = list(encoding_test.keys())[rand1]
  image = encoding_test[pic].reshape((1,2048))
  print(os.path.join(root_captioning,'Flicker8k_Dataset', pic))
  p=pic.split('.')[0]
  ref_list = []
  ref_list.append(pre_processed_captions[p])
  x=plt.imread(os.path.join(root_captioning,'Flicker8k_Dataset', pic))
  plt.imshow(x)
  plt.show()
  k=[]
  k.append(generateCaption(image))
  bleu_1 = nltk.translate.bleu_score.corpus_bleu(ref_list, k, weights=(1, 0, 0, 0))
  bleu_2 = nltk.translate.bleu_score.corpus_bleu(ref_list, k, weights=(0, 1, 0, 0))
  bleu_3 = nltk.translate.bleu_score.corpus_bleu(ref_list, k, weights=(0, 0, 1, 0))
  bleu_4 = nltk.translate.bleu_score.corpus_bleu(ref_list, k, weights=(0, 0, 0, 1))
  print("Generated Caption:",k)
  print("Original Caption:")
  print(pre_processed_captions[p])
  print("BLEU-1\t", bleu_1)
  print("BLEU-2\t", bleu_2)
  print("BLEU-3\t", bleu_3)
  print("BLEU-4\t", bleu_4)
  print("_____________________________________")

NameError: name 'randint' is not defined

IndexError: ignored

In [1]:
for z in range(300): # set higher to see more examples
  rand1=randint(1,1000)
  pic = list(encoding_test.keys())[rand1]
  image = encoding_test[pic].reshape((1,2048))
  print(os.path.join(root_captioning,'Flicker8k_Dataset', pic))
  p=pic.split('.')[0]
  ref_list = []
  ref_list.append(pre_processed_captions[p])
  x=plt.imread(os.path.join(root_captioning,'Flicker8k_Dataset', pic))
  plt.imshow(x)
  plt.show()
  k=[]
  k.append(generateCaption(image))
  bleu_1 = nltk.translate.bleu_score.corpus_bleu(ref_list, k, weights=(1, 0, 0, 0))
  bleu_2 = nltk.translate.bleu_score.corpus_bleu(ref_list, k, weights=(0, 1, 0, 0))
  bleu_3 = nltk.translate.bleu_score.corpus_bleu(ref_list, k, weights=(0, 0, 1, 0))
  bleu_4 = nltk.translate.bleu_score.corpus_bleu(ref_list, k, weights=(0, 0, 0, 1))
  print("Generated Caption:",k)
  print("Original Caption:")
  print(pre_processed_captions[p])
  print("BLEU-1\t", bleu_1)
  print("BLEU-2\t", bleu_2)
  print("BLEU-3\t", bleu_3)
  print("BLEU-4\t", bleu_4)
  print("_____________________________________")

NameError: name 'randint' is not defined

In [2]:
pre_processed_captions['2435685480_a79d42e564']

NameError: name 'pre_processed_captions' is not defined